## `*Importing required packages*`  

In [ ]:
import numpy as np
import math,cv2,os
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
#from keras.backend.tensorflow_backend import set_session
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
#config = tf.compat.v1.configProto(device_count = {'GPU' : 0, 'CPU' : 56})
sess = tf.compat.v1.Session(config = config)
from keras import backend as K
#tf.compat.v1.keras.backend.get_session(sess)

# Data *Input*

In [ ]:
train_dataset_path = "Train/"
test_dataset_path = "Test/"
in_dir = 'Data/'
Categories = ['One', 'Two']


## Train & Test Data Generator  

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(in_dir+train_dataset_path,
target_size = (64,64), 
class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(in_dir+test_dataset_path,
target_size = (64,64),
batch_size = 32,
color_mode = 'rgb',
shuffle = True,
seed = None,
class_mode = 'categorical')

## Architecture of CNN model

In [ ]:
INPUT_SHAPE = (64, 64, 3) # change to (SIZE, SIZE, 3)
inp = keras.layers.Input(shape=INPUT_SHAPE)
conv1 = keras.layers.Conv2D(32, kernel_size=(3, 3),
 activation='relu', padding='same')(inp)
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
norm1 = keras.layers.BatchNormalization(axis=-1)(pool1)
drop1 = keras.layers.Dropout(rate=0.3)(norm1)

conv2 = keras.layers.Conv2D(32, kernel_size=(3, 3),
 activation='relu', padding='same')(drop1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
norm2 = keras.layers.BatchNormalization(axis=-1)(pool2)
drop2 = keras.layers.Dropout(rate=0.3)(norm2)
flat = keras.layers.Flatten()(norm2) # Flatten the matrix to get it ready for dense.
hidden1 = keras.layers.Dense(512, activation='relu')(flat)
norm3 = keras.layers.BatchNormalization(axis=-1)(hidden1)
drop3 = keras.layers.Dropout(rate=0.3)(norm3)

hidden2 = keras.layers.Dense(256, activation='relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis=-1)(hidden2)
drop4 = keras.layers.Dropout(rate=0.3)(norm3)
out = keras.layers.Dense(2, activation='softmax')(drop4) 

### Model summery 

In [ ]:
model = keras.Model(inputs=inp, outputs=out)
model.compile(optimizer='adam',
loss='categorical_crossentropy', # Check between binary_crossentropy and categorical_crossentropy
 metrics=['accuracy'])

### Training Model

In [ ]:
history = model.fit(training_set,
 validation_data = test_set,
 batch_size=64,
 verbose=1,
 epochs=20,
 shuffle=True
# callbacks=callbacks
)


### mModel evaluation

In [ ]:
score = model.evaluate(training_set,verbose=1)
print("Testing loss: ",score[0])
print("Testing Accuracy :", score[1]*100)


f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('CNN Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

max_epoch = len(history.history['accuracy']) + 1
epoch_list = list(range(1, max_epoch))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(1, max_epoch, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(1, max_epoch, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")
plt.show()